In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,dataset
from torch.utils.data import sampler,TensorDataset
import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import transforms
import cv2
import os
dtype = torch.float32
print("CUDA: ",torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%load_ext autoreload
%autoreload 2
%matplotlib inline
root = "/home/cth/code/toys/train"

class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset.
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start=0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples

class DogSet(dataset.Dataset):
    def __init__(self, path, mode = 'train', transforms_=None):
        self.t = transforms.Compose(transforms_)
        self.path = path
        self.data = pd.read_csv(os.path.join(path,'labels.csv'))
        self.mode = mode 
        c = sorted(self.data['breed'].unique())
        self.breed_dict = {c[i]:i for i in range(len(c))} 
        
    def __getitem__(self,index):
        label = self.data.iloc[index]['breed']
        img = cv2.imread(f'{self.path}/{self.mode}/{self.data.iloc[index]["id"]}.jpg')
        img = self.t(img)
        return [img, self.breed_dict[label]]
    
    def __len__(self):
        return len(self.data)

transform_ = [
            transforms.ToTensor(),
            transforms.Resize((224,224)),
         ]

dataset = DogSet('/home/cth/code/toys','train', transform_)
batch_size = 64
loader_train = DataLoader(dataset, batch_size = batch_size,sampler=ChunkSampler(10000,0))
loader_val = DataLoader(dataset, batch_size = batch_size,sampler=ChunkSampler(len(os.listdir(root))-10001,10000))
 

CUDA:  True
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from torchvision.models import resnet152
model_res = resnet152(pretrained=True)
for param in model_res.parameters():
    param.requires_grad = False
model_res_fc_infeature = model_res.fc.in_features
print(model_res_fc_infeature)
model_res.fc = nn.Linear(model_res_fc_infeature,1024)
model = nn.Sequential(
    model_res,
    nn.ReLU(),
    nn.Linear(1024,120,True)
)
data = next(enumerate(loader_train))[-1][0].type(dtype)
print(model(data).shape)

2048
torch.Size([64, 120])


In [13]:
def check_accuracy_part34(loader, model):
    # if not loader.dataset.train:
    #     print('Checking accuracy on validation set')
    # else:
    #     print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

def train_part34(model, optimizer, loss_func , epochs=1 , print_every=100):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            #print("scores:",scores)
            #print("y:",y)
            loss = loss_func(scores, y)
            #loss = F.softmax(scores,y)
            #print("loss:",loss)
            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()
            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()
            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                #print(x.shape)
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                print("check on val:",end="")
                check_accuracy_part34(loader_val, model)
                print("check on train:",end="")
                check_accuracy_part34(loader_train,model)
                print()
                

In [14]:
optimizer = torch.optim.Adam((para for para in model.parameters() if para.requires_grad),lr=0.004)
train_part34(model, optimizer, torch.nn.CrossEntropyLoss(),100,100)

torch.Size([64, 3, 224, 224])
Iteration 0, loss = 4.7746
check on val:Got 2 / 221 correct (0.90)
check on train:

KeyboardInterrupt: 